In [66]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'circular economy waste household' # maybe add 'trash' 'garbage' 'individual'
}


In [67]:
##GET TITLE/LINK/AUTHORS OF N*10 ARTICLES

#loop through pages for n pages
n = 10

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns = ['titles', 'links', 'authors'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #locate and strip the '[HTML]' or '[PDF]' in the titles
    titles = [title[:title.find('[')-1] if title.find('[') != -1 else title for title in titles]

    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'titles' : titles, 'links' : links, 'authors' : authors}))


  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
results.to_csv('article_info2.csv')

In [5]:
## BASIC CODE TO SCRAPE ABSTRACT 

import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [6]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [200]>


In [50]:
## CODE TO DOWNLOAD 1 LINK

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [45]:
#scraping function

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome() #initialize web-finder
    main_page_url = 'https://sci-hub.ru/' #open sci-hub
    driver.get(main_page_url)
    time.sleep(1)
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)
    
    try:
        # Find download link
        download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
        onclick_attribute = download_button.get_attribute('onclick')
    except:
        driver.quit()
        return url_to_search

    
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    download_link = 'https://sci-hub.ru' + pdf_url
    # Download the PDF using requests
    response = requests.get(download_link)
    with open(f'pdfs/{file_name}.pdf', 'wb') as file:
       file.write(response.content)
    
    driver.quit()

    return None



In [68]:
##ALTERNATIVE FUNCTION THAT ONLY GETS LINKS
def pdf_download_2(url_to_search, file_name):
    driver = webdriver.Chrome()  # Initialize web-finder
    main_page_url = 'https://sci-hub.ru/'  # Open sci-hub
    driver.get(main_page_url)
    time.sleep(1)
    
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)
    
    download_links = []  # List to store download links
    
    try:
        # Find all download buttons
        download_buttons = driver.find_elements(by=By.XPATH, value="//button[@onclick]")
        
        for download_button in download_buttons:
            onclick_attribute = download_button.get_attribute('onclick')
            pdf_url = onclick_attribute.split("'")[1]
            download_link = 'https://sci-hub.ru' + pdf_url
            download_links.append(download_link)
            
    except:
        driver.quit()
        return [url_to_search]  # Return the original URL in case of an exception
    
    driver.quit()
    
    return download_links


In [71]:
##DOWNLOADING DOI LINKS
import requests
from bs4 import BeautifulSoup
import re

def extract_doi_from_webpage(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            doi_elements = soup.find_all(string=re.compile(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b'))# finds elements containing DOI information
            
            if doi_elements:
                dois = []
                for element in doi_elements:# extracts DOIs from matched elements
                    match = re.findall(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b', str(element))
                    if match:
                        dois.extend(match)
                    response = requests.get(doi_elements)
                    with open(f'PDFS/{file_name}.pdf', 'wb') as file:
                        file.write(response.content)
                
                return dois  # return list of extracted DOIs
            else:
                return None
        else:
            print(f"Failed to fetch content from {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching content from {url}: {e}")
        return None

file_name = 'dois'
download_links = pdf_download_2(url_to_search, file_name)

# Extract DOIs from each URL in the list
for url in download_links:
    if not url.startswith('http://') and not url.startswith('https://'):
        url = 'https://' + url
    extracted_dois = extract_doi_from_webpage(url)
    if extracted_dois:
        print(f"Extracted DOIs from {url}: {extracted_dois}")
    else:
        print(f"No DOIs found on the webpage {url}")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF772A24D02+56194]
	(No symbol) [0x00007FF7729904B2]
	(No symbol) [0x00007FF7728376AA]
	(No symbol) [0x00007FF772810AFD]
	(No symbol) [0x00007FF7728AC9AB]
	(No symbol) [0x00007FF7728C201F]
	(No symbol) [0x00007FF7728A5C23]
	(No symbol) [0x00007FF772874A45]
	(No symbol) [0x00007FF772875AD4]
	GetHandleVerifier [0x00007FF772D9D5BB+3695675]
	GetHandleVerifier [0x00007FF772DF6197+4059159]
	GetHandleVerifier [0x00007FF772DEDF63+4025827]
	GetHandleVerifier [0x00007FF772ABF029+687785]
	(No symbol) [0x00007FF77299B508]
	(No symbol) [0x00007FF772997564]
	(No symbol) [0x00007FF7729976E9]
	(No symbol) [0x00007FF772988094]
	BaseThreadInitThunk [0x00007FFDCCF87344+20]
	RtlUserThreadStart [0x00007FFDCDD626B1+33]


In [22]:
##to call on all links !!! DO NOT RERUN, IT WILL SET LIST BACK TO NONE
#RUN
import pandas as pd
import os
from tqdm.notebook import tqdm

data = pd.read_csv('article_info2.csv')
title = data['titles']
links = data['links']
author = ['authors'] 


def file_exists(file_path):
    return os.path.exists(file_path)

list_error_links = []


In [32]:
#function to call on all links
#RUN
for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'
    if file_exists(pdf_file_path) is True:
        print("PDF already downloaded")
    else:
        if row['links'] in list_error_links:
            print('error already encountered')
        else:
            url_returned = pdf_download(row['links'], index)  #(links[index], index)
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'ERROR FOR {url_returned}')

PDF already downloaded
error already encountered
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
error already encountered
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
PDF already downloaded
error already encountered
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
PDF already downloaded
error already encountered
PDF already downloaded
PDF already downloaded
error already encountered
PDF already downloaded
PDF already downloaded
PDF already downloaded
error already encountered


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="request"]"}
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF772A24D02+56194]
	(No symbol) [0x00007FF7729904B2]
	(No symbol) [0x00007FF7728376AA]
	(No symbol) [0x00007FF7728816D0]
	(No symbol) [0x00007FF7728817EC]
	(No symbol) [0x00007FF7728C4D77]
	(No symbol) [0x00007FF7728A5EBF]
	(No symbol) [0x00007FF7728C2786]
	(No symbol) [0x00007FF7728A5C23]
	(No symbol) [0x00007FF772874A45]
	(No symbol) [0x00007FF772875AD4]
	GetHandleVerifier [0x00007FF772D9D5BB+3695675]
	GetHandleVerifier [0x00007FF772DF6197+4059159]
	GetHandleVerifier [0x00007FF772DEDF63+4025827]
	GetHandleVerifier [0x00007FF772ABF029+687785]
	(No symbol) [0x00007FF77299B508]
	(No symbol) [0x00007FF772997564]
	(No symbol) [0x00007FF77299

In [33]:
##DOWNLOADING ERROR LINKS
import requests
from bs4 import BeautifulSoup
import re

def extract_doi_from_webpage(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        doi_elements = soup.find_all(string=re.compile(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b'))# finds elements containing DOI information
        
        if doi_elements:
            dois = []
            for element in doi_elements:# extracts DOIs from matched elements
                match = re.findall(r'\b(10\.\d{4,}(?:\.\d+)*/\S+)\b', str(element))
                if match:
                    dois.extend(match)
            
            return dois  # return list of extracted DOIs
        else:
            return None
    else:
        print(f"Failed to fetch content from {url}")
        return None

list_error_links 

# Extract DOIs from each URL in the list
for url in list_error_links:
    extracted_dois = extract_doi_from_webpage(url)
    if extracted_dois:
        print(f"Extracted DOIs from {url}: {extracted_dois}")
    else:
        print(f"No DOIs found on the webpage {url}")

Failed to fetch content from https://www.sciencedirect.com/science/article/pii/S0892687522002710
No DOIs found on the webpage https://www.sciencedirect.com/science/article/pii/S0892687522002710
Failed to fetch content from https://www.sciencedirect.com/science/article/pii/S0959652622040185
No DOIs found on the webpage https://www.sciencedirect.com/science/article/pii/S0959652622040185
No DOIs found on the webpage http://ppeu.stu.cn.ua/article/view/126051
Extracted DOIs from https://www.mdpi.com/2079-9276/9/11/128: ['10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128', '10.3390/resources9110128']
Extracted DOIs from https://www.emerald.com/insight/content/doi/10.1108/MEQ-04-2021-0089/full/html: ['10.1108/MEQ-04-2021-0089/full/html', '10.1108/MEQ-04-2021-0089/full/html', '10.1108/MEQ-04-2021-0089']
Extracted DOIs from ht

Scrape Discussion

In [ ]:
%pip install pypdf

   ---------------------------------------- 0.0/277.6 kB ? eta -:--:--
   ---------------------------------------- 277.6/277.6 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.getcwd()

'c:\\Users\\sabri\\trashmasters\\scholar scraper'

In [2]:
from pypdf import PdfReader
from os import listdir
from os.path import isfile, join
import os
from tqdm.notebook import tqdm

mypath = 'C:\\Users\\sabri\\Desktop\\pdfs'

pdf_folder_content = [f for f in listdir(mypath)]

# print(f'Found {len(pdf_folder_content)} PDFs in folder!')
print(pdf_folder_content)

all_pdf_texts = []
discussion_text = ''

for pdf in pdf_folder_content:
    print(f'Analyzing {pdf}...')

    try:
       reader = PdfReader(f'{mypath}\\{pdf}')
    except:
        print(f'Error reading {pdf}!')
        continue

    n_pages = len(reader.pages)
	
    print(f'{pdf} has {n_pages} pages!')

    all_text = []

    for i in range(0,n_pages):  
      page_text = reader.pages[i].extract_text()
      print(page_text)
      # all_text.append(page_text)
      
      with open(f'{mypath}\\{pdf}.txt', 'a') as f:
        f.write(f'NEW PAGE: {i}/{n_pages}' + page_text)

    result_string = ' '.join(all_text)
    all_pdf_texts.append(result_string)

##SWITCH TO A DICTIONARY INSTEAD OF A LIST

['0.pdf', '0.pdf.txt', '0_discussion.txt', '11.pdf', '11.pdf.txt', '14.pdf', '14.pdf.txt', '15.pdf', '15.pdf.txt', '16.pdf', '16.pdf.txt', '16_discussion.txt', '17.pdf', '17.pdf.txt', '17_discussion.txt', '19.pdf', '19.pdf.txt', '20.pdf', '22.pdf', '22.pdf.txt', '22_discussion.txt', '23.pdf', '23.pdf.txt', '23_discussion.txt', '24.pdf', '24.pdf.txt', '27.pdf', '27.pdf.txt', '29.pdf', '3.pdf', '3.pdf.txt', '30.pdf', '30.pdf.txt', '35.pdf', '35.pdf.txt', '37.pdf', '37.pdf.txt', '38.pdf', '39.pdf', '3_discussion.txt', '4.pdf', '41.pdf', '41.pdf.txt', '42.pdf', '46.pdf', '47.pdf', '48.pdf', '58.pdf', '59.pdf', '61.pdf', '8.pdf', '8.pdf.txt', '8_discussion.txt', '9.pdf']
Analyzing 0.pdf...
0.pdf has 12 pages!
Contents lists available at ScienceDirect
Ecological Economics
journal homepage: www.elsevier.com/locate/ecolecon
Analysis
Consumers in a Circular Economy: Economic Analysis of Household Waste
Sorting Behaviour
Doan Nainggolana,⁎, Anders Branth Pedersena, Sinne Smedb, Kahsay Haile Zemo

invalid pdf header: b'NEW P'


segment1,theimportanceofrecyclingcapacityismorepronouncedin
determiningthepreferencesforwastesortingschemesbyhouseholdsin
segment 2. This indicates that segment 2 finds it important that the
municipality actually has the capacity to utilise the sorted waste. We
identify households in segment 2 as ‘the sceptics’ (S2).
Segment3coversapproximately15%ofthesurveyrespondents.In
general, of all the segments identified through LCM, respondents in
segment3demonstratethemostpositiveviewonsortingofplasticsand
bio-waste, and at the same time they reveal a clear preference for the
possibilitytodisposehazardouswastetoallocatedcontainerwithinthe
vicinity of their residences. The parameter estimates for the attributes
hazardous waste, plastic and bio-waste are statistically significant and
positive (see Table 4). Interestingly, in contrast to the other segments,
the parameter estimate for recycling capacity is statistically non-sig-
nificantforsegment3whichmaysuggestthatthepreferencesforwaste
sorting 

EOF marker not found
invalid pdf header: b'using'
EOF marker not found


Error reading 0.pdf.txt!
Analyzing 0_discussion.txt...
Error reading 0_discussion.txt!
Analyzing 11.pdf...
11.pdf has 12 pages!
Urban waste ﬂows and their potential for a circular economy model at
city-region level
Vanessa Zellera,⇑, Edgar Towaa,b, Marc Degrezb, Wouter M.J. Achtena,⇑
aInstitute for Environmental Management and Land-use Planning, Université Libre de Bruxelles (ULB), Av. F.D. Roosevelt 50, 1050 Brussels, Belgium
b4MAT Department, Université Libre de Bruxelles (ULB), Av. F.D. Roosevelt 50, 1050 Brussels, Belgium
article info
Article history:
Received 11 June 2018
Revised 28 September 2018Accepted 20 October 2018
Keywords:
Waste input-outputWaste supply and useUrban waste metabolismCircular economyWaste ﬂow analysis
Waste performanceabstract
To enable cities to become more circular, i.e. close material cycles, decision-makers need detailed data
about the production and treatment of waste. At city level, conventional statistics on waste are oftenincomplete or lack detail. W

...

Declaration of Competing Interest
The authors declare that they have no known competing ﬁnan-
cial interests or personal relationships that could have appeared
to inﬂuence the work reported in this paper.
Acknowledgements
The work of the authors was partially supported by different
funding sources, i.e., MSCA-IF-2018 - Individual Fellowships
(inTECH, Grant agreement ID: 841193) within H2020-EU.1.3.2. -
Nurturing excellence by means of cross-border and cross-sector
mobility and by funds of the Federal Ministry of Education and
Research (BMBF) based on a decision of the Parliament of the Fed-
eral Republic of Germany via the Project Management Jülich (PtJ)
within the joint research project Agrarsysteme der Zukunft: F4F -
Nahrung der Zukunft, Teilprojekt I (Grant no. 031B0730 I).
References
Ahuja, K., Mamtani, K., 2020. Edible Insects Market size to exceed $1.5bn by 2026.
Global Market Insights Inc. .
Alattar, M.A., Alattar, F.N., Popa, R., 2016. Effects of microaerobic fermentation and
b

invalid pdf header: b'NEW P'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<html'
EOF marker not found
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


podaci-o-komunalnom . (Accessed 28 December 2019).
HGK (Croatian Chamber of Commerce), 2019. Trinaesta konferencija o sigurnosti i
kvaliteti hrane. https://www.hgk.hr/13-konferencija-o-sigurnosti-i-kvaliteti-
hrane-najava . (Accessed 2 June 2019).
Hogg, D., 2002. Costs for municipal waste managent in the EU, ﬁnal report to DG
environment. ec.europa.eu/environment/waste/studies/pdf/eucostwaste.pdf .
JRC EC, 2011. Supporting environmentally sound decisions for biowaste manage-
ment. https://ec.europa.eu/jrc/en/publication/eur-scienti ﬁc-and-technical-
research-reports/supporting-environmentally-sound-decisions-biowaste-
management-practical-guide-life-cycle accessed 15 3 October 2019.
Kumar Ghosh, S., Di Maria, F., 2018. A comparative study of issues, challenges and
strategies of biowaste managenent in India and Italy. Detritus 1, 8 e17.https://
doi.org/10.26403/detritus/2018.8 .
Larsen, A.W., Merrild, H., Møller, J., Christensen, T.H., 2010. Waste collection systems
for recyclables: an 

invalid pdf header: b'NEW P'
EOF marker not found
invalid pdf header: b'they '
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Robert, J., Kubler, S., Kolbe, N., Cerioni, A., Gastaud, E., Fr €amling, K., 2017. Open IoT
Ecosystem for Enhanced Interoperability in Smart Cities dExample of
M/C19etropole De Lyon. Sensors 17 (12), 2849 .
Rousta, K., Bolton, K., Lundin, M., Dahl /C19en, L., 2015. Quantitative assessment of dis-
tance to collection point and improved sorting information on source separa-
tion of household waste. Waste Manag. 40, 22 e30.
Sekito, T., Prayogo, T.B., Dote, Y., Yoshitake, T., Bagus, I., 2013. In ﬂuence of a
community-based waste management system on people ’s behavior and waste
reduction. Resour. Conserv. Recycl. 72, 84 e90.
Spence, L.J., Rinaldi, L., 2014. Governmentality in accounting and accountability: A
case study of embedding sustainability in a supply chain. Account. Org. Soc. 39
(6), 433 e452.
Stoeva, K., Alriksson, S., 2017. In ﬂuence of recycling programmes on waste sepa-
ration behaviour. Waste Manag. 68, 732 e741, 2017 .
Stojkoska, B.L.R., Trivodaliev, K.V., 2017. A review of I

In [4]:
print(all_pdf_texts)
text = all_pdf_texts[0].split(' ')

# replace \n with space, then split it by the space
text = [word.replace('\n', ' ') if '\n' in word else word for word in text]
text = ' '.join(text).split(' ')

count = 0

# split, then join
# filter after tokenise then filter by list comprehension
#l for l in list, if l = discussioin etc


# delete all elements before the first one that contains the word "discussion"
for word in text:
    if 'Discussion' in word:
        text = text[count + 1:]
        break
    
    if 'Conclusion' in word:
        text = text[:count]
        break
    
    count += 1

text = ' '.join(text).split('. ')

with open('text.txt', 'w') as f:
    f.write('.'.join(text))


print(text)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['']


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re


from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel

lemmatizer = WordNetLemmatizer()

additional_stopwords = ['et', 'al','b', 'fw', 'e'] 

def preprocess_text(text):
    #YOUR CODE HERE

    # removes special characters and punctuation (STEP 1: Text cleaning)
    text = re.sub(r'[^A-Za-z\s]', '', text) 
    # remove additional white spaces (STEP 1: Text cleaning)
    text = re.sub(r'\s+', ' ', text).strip() 

    #put text in lowercases (STEP 2: Case normalization)
    text = text.lower()

    #STEP 3: tokenization
    tokens = word_tokenize(text)

    #STEP 4: removing stopwords:
    stopwords_list = set(stopwords.words('english')).union(additional_stopwords)
    tokens = [token for token in tokens if token not in stopwords_list]

    #STEP 5: lemmeatization: 
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_words

In [ ]:
processed_text = [preprocess_text(text) for text in all_pdf_texts]

corpus = corpora.Dictionary(processed_text)
doc_term_matrix = [corpus.doc2bow(text) for text in processed_text]

num_topics = 45

lda_model = LdaModel(doc_term_matrix, id2word = corpus, num_topics = num_topics)



In [ ]:
from collections import Counter

document_topics = [lda_model.get_document_topics(doc) for doc in doc_term_matrix]
all_topics = [topic[0] for topics in document_topics for topic in topics]

topics_nb = Counter(all_topics)

most_nb_topics = [topic for topic, count in topics_nb.most_common()]
print("The most common topics and the number of instances are:")
for topic, x in topics_nb.most_common():
    print(f"Topic {topic +1}: Count {x}")
for topic in most_nb_topics:
    topics_words = lda_model.show_topic(topic)
    most_words = [word for word, _ in topics_words]
    print(f"Most used words in Topic {topic +1}: {', '.join(most_words)}")

The most common topics and the number of instances are:
Topic 16: Count 6
Topic 44: Count 4
Topic 9: Count 3
Topic 29: Count 3
Topic 2: Count 2
Topic 43: Count 2
Topic 31: Count 1
Topic 21: Count 1
Topic 38: Count 1
Topic 6: Count 1
Topic 10: Count 1
Topic 5: Count 1
Most used words in Topic 16: waste, plastic, household, recycling, food, economy, product, collection, system, material
Most used words in Topic 44: waste, recycling, household, plastic, collection, product, ewaste, study, system, business
Most used words in Topic 9: waste, plastic, recycling, material, business, household, food, study, packaging, korea
Most used words in Topic 29: waste, household, food, recycling, consumer, product, plastic, collection, study, material
Most used words in Topic 2: waste, food, product, recycling, consumer, ewaste, household, study, collection, system
Most used words in Topic 43: waste, recycling, system, product, ewaste, household, study, consumer, food, management
Most used words in Topi

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Function to compute coherence score
def compute_coherence(dictionary, corpus, texts, limit, start=2, step=3):
    # YOUR CODE HERE
    coherence_values = []
    for num_topics in range(start, limit, step):
        lda_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = num_topics, random_state = 100)

        coherence_model_lda = CoherenceModel(model = lda_model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherence_model_lda.get_coherence())
    
    return coherence_values

coherence_values = compute_coherence(dictionary=corpus, corpus= doc_term_matrix, texts=processed_text, limit=50, start = 5, step=5)

optimal_nb_topics = (coherence_values.index(max(coherence_values)) + 1) * 5
print(f"the optimal number of topic for this model is: {optimal_nb_topics}")

the optimal number of topic for this model is: 45
